In [1]:
# insert matplotlib in colab
%matplotlib inline

In [5]:
import torch
from torch import  nn
from torch.utils.data import  DataLoader
# als many domian specific lib,like torchaudio, torchtext
from torchvision import datasets
# datasets: CIFAR, COCO, etc
# also torchvision.datasets include 2 argument: transform, taget_transform to modify the samples
from torchvision.transforms import ToTensor

In [10]:
#Dowload training data from open datasets
#!datasets.FashionMNIST?
training_data = datasets.FashionMNIST(
   root = "data",
   train = True,
   download = True,
   transform = ToTensor
)

test_data = datasets.FashionMNIST(
   root = "data",
   train = False,
   download = True,
   transform = ToTensor
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [18]:
#pass the data to Dataloader
#Dataloader wraps an iterable over our dataset, and support automatic batching, sampling, shulffing and multiprocessing data loading
#!DataLoader?
#batch size is number of samples processed before model updated
#small batch size, need more epoched to convergent, large may miss the local lowest point.
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(dataset=training_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

""" for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break """

' for X, y in test_dataloader:\n    print(f"Shape of X [N, C, H, W]: {X.shape}")\n    print(f"Shape of y: {y.shape} {y.dtype}")\n    break '

In [19]:
# Get cpu or gpu device for training.
# mps: meta performance shaders for macos with Apple silicon
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
# Inherits from nn.Module
# nn.Module: base class for all nn module, your model should also subclass this class, nest them in a tree structure
class NeuralNetwork(nn.Module):
    #define the layers in NN
    def __init__(self):
        #super() with __init__()
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    #define how the data pass the model
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


address of self =  140360232863760
address of obj =  140360232863760
